In [78]:
# imports
from bs4 import BeautifulSoup
from bs4.element import ResultSet
import requests
import json
import re

In [79]:
session = requests.Session()
# Handles Cookies for logging in and persistance

url = "https://homeaccess.beth.k12.pa.us/HomeAccess/Account/LogOn/index.html"
# URL for going to the site

r = session.get(url)
# setting the website url to 'r'

soup = BeautifulSoup(r.text, features="html.parser")
# extracting the raw website data into 'soup'

VerificationToken = soup.find("input", {"name":"__RequestVerificationToken"}).get("value")
# Retarted Verification Token added for no good reason

payload = {
    "__RequestVerificationToken" : VerificationToken,
    "ReturnUrl" : "/HomeAccess/Classes/Classwork",
    "SCKTY00328510CustomEnabled" : False,
    "Database" : 10,
    "LogOnDetails.UserName" : "mooren",
    "LogOnDetails.Password" : "AlphaBeta4721",
}
# Payload Information to send for login

GradeLogin = session.post(url = "https://homeaccess.beth.k12.pa.us/HomeAccess/Account/LogOn/index.html", data = payload)
# Sending to Payload to the website for login

soup = BeautifulSoup(GradeLogin.text, features="html.parser")
# Parsing the response from the payload

assignments = soup.find("iframe")
# Finding the Location of the Grades

link_get_source = "https://homeaccess.beth.k12.pa.us" + assignments.get("src")
# Generates Link for the Extraction

GradesLocation = session.post(url = link_get_source, data = payload)

In [84]:
souped = BeautifulSoup(GradesLocation.text, features="html.parser")
# print(GradesLocation.text)
hiddenInputs = souped.select("form#fmMain > input")
# print(hiddenInputs)
payload = {x["id"]:x["value"] for x in hiddenInputs}
hiddenInputs = souped.select("form#fmMain > div > input")
payload.update({x["id"]:x["value"] for x in hiddenInputs})
payload["__EVENTTARGET"] = "ctl00$plnMain$btnRefreshView"
# print(payload)

In [81]:
payload["ctl00$plnMain$ddlReportCardRuns"] = "2-2023"
newLoc = session.post(url = link_get_source, data = payload)

In [82]:
soop = BeautifulSoup(newLoc.text, features="html.parser")

classes = soop.find_all("a", {"class":"sg-header-heading"})

averages = list()
count = 0
for i in enumerate(classes):
    x = soop.find("span", {"id":f'plnMain_rptAssigmnetsByCourse_lblOverallAverage_{i[0]}'})
    averages.append(x)

In [83]:
print(averages)

[None, None, <span id="plnMain_rptAssigmnetsByCourse_lblOverallAverage_2">78.67</span>, None]
